# HuggingFace TrieGuided Decoding

In [1]:
# %load_ext autoreload
# %autoreload 2

torch_device = 'cpu'

In [2]:
!which python

/Users/fuad/opt/miniconda3/envs/lexflow/bin/python


In [3]:
!pip install outlines -q
!pip install -q transformers
!pip install -q openai
!pip install -q datasets
!pip install -q accelerate
!pip install trieregex

In [4]:
import site
import os
import transformers  # replace 'my_library' with the actual library name

In [5]:
library_path = os.path.dirname(transformers.__file__)
print(library_path)


/Users/fuad/opt/miniconda3/envs/lexflow/lib/python3.10/site-packages/transformers


In [6]:
import sys

print(sys.path)

['/Users/fuad/Documents/projects/lexflow/src/notebooks', '/Users/fuad/Documents/projects/lexflow', '/Users/fuad/opt/miniconda3/envs/lexflow/lib/python3.10/site-packages/transformers', '/Users/fuad/opt/miniconda3/envs/lexflow/lib/python310.zip', '/Users/fuad/opt/miniconda3/envs/lexflow/lib/python3.10', '/Users/fuad/opt/miniconda3/envs/lexflow/lib/python3.10/lib-dynload', '', '/Users/fuad/opt/miniconda3/envs/lexflow/lib/python3.10/site-packages']


##  Exploring api

In [7]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Open-Orca/Mistral-7B-OpenOrca
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


/Users/fuad/opt/miniconda3/envs/lexflow/lib/python3.10/site-packages/beartype/_util/module/utilmodimport.py:149: BeartypeModuleUnimportableWarning: Ignoring module "onnx" importation exception:
	ImportError: attempted relative import beyond top-level package
  warn(


In [8]:
model.generation_config

GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256
}

In [9]:
from custom_utils import CustomLogitsProcessor

In [10]:
from transformers.generation.logits_process import LogitsProcessorList

logit_processors = LogitsProcessorList([CustomLogitsProcessor()])

In [11]:
input_ids = tokenizer('I enjoy walking with my cute dog', return_tensors='pt')  #.to(torch_device)

In [20]:
out = model.generate(**input_ids, 
               max_new_tokens=40,
               do_sample=True,
               top_k=50,
               # num_beams=5,
               early_stopping=True,
               # logits_processor=logit_processors
               )

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [24]:
print(out.shape)
print(tokenizer.decode(out[0], skip_special_tokens=True))

torch.Size([1, 47])
I enjoy walking with my cute dog and I like seeing it in a variety of different situations and I think it would have been much better if I could stay outside a lot of time. My dog has never had a break but I like


## Building document index 

### trie
this one seems to work as i want but its memory intensive. can optimise later

In [25]:
from custom_utils import read_document
document = read_document('document.txt')

In [26]:
len(document)

5025

In [40]:
documents = [document]

In [41]:
from custom_utils import DocumentIndex

In [43]:
document_index = DocumentIndex(documents, tokenizer)


([',', 'Ġ-', 'Ġwas', 'Ġsaid'], 'Mrs Braverman,')

In [58]:
document_index.get_next_words(" Mr Sunak's")

{'next_words': {' cabinet': 1},
 'next_ids': {13447: 1},
 'full_text_list': [" Mr Sunak's cabinet"]}

In [61]:
document_index.get_next_words("Mr Sunak")

{'next_words': {' had': 1},
 'next_ids': {550: 1},
 'full_text_list': ['Mr Sunak had']}

In [62]:
document_index.get_next_words(" Mrs Braverman.")

{'next_words': {'\n': 1},
 'next_ids': {198: 1},
 'full_text_list': [' Mrs Braverman.\n']}

In [66]:
document_index.get_next_words("Mrs Braverman.")

{'next_words': {}, 'next_ids': {}, 'full_text_list': []}

In [64]:
document_index.get_next_words("Mrs Braverman")

{'next_words': {',': 1, ' -': 1, ' was': 1, ' said': 2},
 'next_ids': {11: 1, 532: 1, 373: 1, 531: 2},
 'full_text_list': ['Mrs Braverman,',
  'Mrs Braverman -',
  'Mrs Braverman was',
  'Mrs Braverman said']}

## Testing on Document

In [45]:
len(document[:512])

512

In [78]:
document

'Suella Braverman accuses Rishi Sunak of betrayal in scathing resignation letter\nPublished\n26 minutes ago\n\nShare\nSuella Braverman and Rishi Sunak\nIMAGE SOURCE,PA MEDIA\nBy Becky Morton & Paul Seddon\nBBC News\nSuella Braverman has launched a full-scale attack on her old boss Rishi Sunak, a day after he sacked her as home secretary.\n\nIn a blistering letter to the prime minister, she said he had repeatedly failed on key policies and broken pledges over immigration.\n\nMr Sunak had adopted "wishful thinking" to "avoid having to make hard choices", she wrote.\n\nAnd she accused him of having "no real intention" of keeping his promises.\n\nADVERTISEMENT\n\nMrs Braverman, a leading figure on the right of the party, claimed she struck a secret deal to serve in Mr Sunak\'s cabinet in exchange for a series of commitments, after Liz Truss\'s premiership imploded last year.\n\nHer support, she added, had been a "pivotal factor" in allowing Mr Sunak to win the support of Tory MPs and enter

In [141]:
def predict(context, question):
    context_ids = tokenizer(context, max_length=450, return_tensors='pt')  #.to(torch_device)
    question_ids = tokenizer(question, max_length=62, return_tensors='pt')  #.to(torch_device)
    
    concatenated_ids = torch.cat((context_ids['input_ids'], question_ids['input_ids']), dim=1)
    concatenated_mask = torch.cat((context_ids['attention_mask'], question_ids['attention_mask']), dim=1)
    input_ids = {'input_ids': concatenated_ids, 'attention_mask': concatenated_mask}
    
    print(input_ids['input_ids'].shape)
    print(input_ids['input_ids'].squeeze())
    print(tokenizer.decode(input_ids['input_ids'][0]))
    out = model.generate(**input_ids, 
               max_new_tokens=40,
               do_sample=True,
               top_k=50,
               # num_beams=5,
               early_stopping=True,
               # logits_processor=logit_processors
               )
    print(out)
    output = tokenizer.decode(out[0], skip_special_tokens=True)
    return output


In [161]:
question = '\n=======\nSuella Braverman accuses Rishi Sunak'

out = predict(document, question)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


torch.Size([1, 465])
tensor([   50,   518,  8466,  9718,   332,   805, 35062,   371, 21644,  3825,
          461,   286, 29355,   287, 44523, 18525,  3850,   198, 24492,   198,
         2075,  2431,  2084,   198,   198, 11649,   198,    50,   518,  8466,
         9718,   332,   805,   290,   371, 21644,  3825,   461,   198,  3955,
        11879,   311, 31033,    11,  4537, 26112,  3539,   198,  3886, 47882,
        35766,  1222,  3362,   311,  6048,   261,   198, 33833,  3000,   198,
           50,   518,  8466,  9718,   332,   805,   468,  5611,   257,  1336,
           12,  9888,  1368,   319,   607,  1468,  6478,   371, 21644,  3825,
          461,    11,   257,  1110,   706,   339, 32564,   607,   355,  1363,
         7705,    13,   198,   198,   818,   257, 40770,   278,  3850,   284,
          262,  6994,  5342,    11,   673,   531,   339,   550,  7830,  4054,
          319,  1994,  4788,   290,  5445, 31241,   625,  6272,    13,   198,
          198,  5246,  3825,   461,   550, 

In [162]:
print(out)

Suella Braverman accuses Rishi Sunak of betrayal in scathing resignation letter
Published
26 minutes ago

Share
Suella Braverman and Rishi Sunak
IMAGE SOURCE,PA MEDIA
By Becky Morton & Paul Seddon
BBC News
Suella Braverman has launched a full-scale attack on her old boss Rishi Sunak, a day after he sacked her as home secretary.

In a blistering letter to the prime minister, she said he had repeatedly failed on key policies and broken pledges over immigration.

Mr Sunak had adopted "wishful thinking" to "avoid having to make hard choices", she wrote.

And she accused him of having "no real intention" of keeping his promises.

ADVERTISEMENT

Mrs Braverman, a leading figure on the right of the party, claimed she struck a secret deal to serve in Mr Sunak's cabinet in exchange for a series of commitments, after Liz Truss's premiership imploded last year.

Her support, she added, had been a "pivotal factor" in allowing Mr Sunak to win the support of Tory MPs and enter No 10.

A No 10 spokesm

In [163]:
question = '\n=======\nIn a list What is Rishi accused of?\n-'
print(predict(document, question))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


torch.Size([1, 466])
tensor([   50,   518,  8466,  9718,   332,   805, 35062,   371, 21644,  3825,
          461,   286, 29355,   287, 44523, 18525,  3850,   198, 24492,   198,
         2075,  2431,  2084,   198,   198, 11649,   198,    50,   518,  8466,
         9718,   332,   805,   290,   371, 21644,  3825,   461,   198,  3955,
        11879,   311, 31033,    11,  4537, 26112,  3539,   198,  3886, 47882,
        35766,  1222,  3362,   311,  6048,   261,   198, 33833,  3000,   198,
           50,   518,  8466,  9718,   332,   805,   468,  5611,   257,  1336,
           12,  9888,  1368,   319,   607,  1468,  6478,   371, 21644,  3825,
          461,    11,   257,  1110,   706,   339, 32564,   607,   355,  1363,
         7705,    13,   198,   198,   818,   257, 40770,   278,  3850,   284,
          262,  6994,  5342,    11,   673,   531,   339,   550,  7830,  4054,
          319,  1994,  4788,   290,  5445, 31241,   625,  6272,    13,   198,
          198,  5246,  3825,   461,   550, 

In [165]:
question = '\n========\nIn a list What did Suela say? \n-'
print(predict(document, question))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


torch.Size([1, 465])
tensor([   50,   518,  8466,  9718,   332,   805, 35062,   371, 21644,  3825,
          461,   286, 29355,   287, 44523, 18525,  3850,   198, 24492,   198,
         2075,  2431,  2084,   198,   198, 11649,   198,    50,   518,  8466,
         9718,   332,   805,   290,   371, 21644,  3825,   461,   198,  3955,
        11879,   311, 31033,    11,  4537, 26112,  3539,   198,  3886, 47882,
        35766,  1222,  3362,   311,  6048,   261,   198, 33833,  3000,   198,
           50,   518,  8466,  9718,   332,   805,   468,  5611,   257,  1336,
           12,  9888,  1368,   319,   607,  1468,  6478,   371, 21644,  3825,
          461,    11,   257,  1110,   706,   339, 32564,   607,   355,  1363,
         7705,    13,   198,   198,   818,   257, 40770,   278,  3850,   284,
          262,  6994,  5342,    11,   673,   531,   339,   550,  7830,  4054,
          319,  1994,  4788,   290,  5445, 31241,   625,  6272,    13,   198,
          198,  5246,  3825,   461,   550, 

# Old

### document graph  - old

this works word by word. it only takes the last word as history 

In [36]:
from custom_utils import build_word_graph
# Example usage
# document = "hello my name is hello his name is ...."
word_graph = build_word_graph(document)

# Now you can query the graph for what comes after "hello"
print(word_graph["Braverman"])  # Outputs: {'my', 'his'}

{'said', 'and', 'for', 'has', 'accuses', '-', 'claim', 'was'}


## testing huggingface - old. now using logits processor

In [64]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Encode the input text
input_text = "The current weather is"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Get the model's output (logits)
with torch.no_grad():
    outputs = model(input_ids)
    predictions = outputs[0]

# Apply softmax to convert logits to probabilities
softmax = torch.nn.Softmax(dim=-1)
probabilities = softmax(predictions)

# Iterate through each word in the input and check the prediction for the next word
for i in range(len(input_ids[0]) - 1):
    # Get the index of the actual next word
    actual_next_word_id = input_ids[0][i + 1].item()
    # Get the probability distribution for the next word
    next_word_probabilities = probabilities[0, i]
    # Get the predicted token ID with the highest probability for the next word
    predicted_next_word_id = next_word_probabilities.argmax().item()

    # Decode the predicted and actual next words
    predicted_token = tokenizer.decode([predicted_next_word_id])
    actual_next_word = tokenizer.decode([actual_next_word_id])

    print(f"Input word: '{tokenizer.decode([input_ids[0][i].item()])}'")
    print(f"Predicted next word: '{predicted_token}'")
    print(f"Actual next word: '{actual_next_word}'")
    print()

# Now handle the last token separately since there's no actual next word in the input
last_word_probabilities = probabilities[0, -1]
predicted_next_word_id_for_last = last_word_probabilities.argmax().item()
predicted_token_for_last = tokenizer.decode([predicted_next_word_id_for_last])

print(f"Input word: '{tokenizer.decode([input_ids[0][-1].item()])}'")
print(f"Predicted next word: '{predicted_token_for_last}'")


Input word: 'The'
Predicted next word: '
'
Actual next word: ' current'

Input word: ' current'
Predicted next word: ' state'
Actual next word: ' weather'

Input word: ' weather'
Predicted next word: ' is'
Actual next word: ' is'

Input word: ' is'
Predicted next word: ' not'


In [65]:
probabilities.shape

torch.Size([1, 4, 50257])

In [66]:
# Get the logits for the last token in the input
last_token_logits = predictions[:, -1, :]

# Define candidate words
candidate_words = ["sunny", "rainy", "cloudy", "windy"]

# Retrieve the indices of the candidate words
candidate_indices = [tokenizer.encode(word)[0] for word in candidate_words]

# Extract the logits for the candidate words
candidate_logits = last_token_logits[:, candidate_indices]

# Find the most likely candidate word
most_likely_word_index = candidate_logits.argmax(-1).item()
predicted_word = candidate_words[most_likely_word_index]

print(predicted_word)

sunny


In [67]:
most_likely_word_index

0

In [68]:
candidate_logits.shape

torch.Size([1, 4])

In [69]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


In [70]:

# Encode the input text
input_text = "Extract from the text what Mrs Braverman said"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Get the model's output (logits)
with torch.no_grad():
    outputs = model(input_ids)
    predictions = outputs[0]

# Apply softmax to convert logits to probabilities
softmax = torch.nn.Softmax(dim=-1)
probabilities = softmax(predictions)

# Now handle the last token separately since there's no actual next word in the input
last_word_probabilities = probabilities[0, -1]
last_token_logits = predictions[:, -1, :]

# Define the trie with potential next words
# trie = # (Assume this is your trie object which has the method search_next_words(sequence))

# Get the possible next words from the trie given the sequence of tokens
sequence_of_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
next_words_from_trie = trie.search_next_words(sequence_of_tokens)
print(next_words_from_trie)
# Convert these possible next words to their corresponding token IDs
next_word_ids_from_trie = {tokenizer.encode(word)[0] for word in next_words_from_trie}

# Mask the probabilities
masked_probabilities = last_word_probabilities.clone()  # Clone the tensor to avoid modifying the original probabilities
for idx in range(masked_probabilities.size(0)):
    if idx not in next_word_ids_from_trie:
        masked_probabilities[idx] = 0

# Renormalize the probabilities
masked_probabilities /= masked_probabilities.sum()

# Find the most likely word from the trie
most_likely_word_id = masked_probabilities.argmax().item()
predicted_word = tokenizer.decode([most_likely_word_id])

print(predicted_word)


{}
!


In [71]:

# Define the function to predict the next word using a trie for restrictions
def predict_next_word(input_text, trie=None, use_trie=True):
    """
    Predicts the next word using GPT-2 model with an option to restrict predictions based on a trie.
    
    Parameters:
    input_text (str): The input text string for which we want to predict the next word.
    trie (Trie object): An optional trie object for restricting predictions.
    use_trie (bool): A flag to determine whether to use the trie for restricting predictions.
    
    Returns:
    str: The predicted next word.
    """
    # Encode the input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Get the model's output (logits)
    with torch.no_grad():
        outputs = model(input_ids)
        predictions = outputs[0]

    # Apply softmax to convert logits to probabilities
    softmax = torch.nn.Softmax(dim=-1)
    probabilities = softmax(predictions)

    # Get the logits and probabilities for the last token in the input
    last_token_logits = predictions[:, -1, :]
    last_word_probabilities = probabilities[0, -1]

    # If the trie is provided and we're using it to restrict predictions
    if trie and use_trie:
        # Get the sequence of tokens from the input text
        sequence_of_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

        # Get the possible next words from the trie given the sequence of tokens
        next_words_from_trie = trie.search_next_words(sequence_of_tokens)

        # Convert these possible next words to their corresponding token IDs
        next_word_ids_from_trie = {tokenizer.encode(word, add_prefix_space=True)[0] for word in next_words_from_trie}

        # Mask the probabilities for tokens not in the trie's next words
        mask = torch.full_like(last_word_probabilities, fill_value=float('-inf'))
        for idx in next_word_ids_from_trie:
            mask[idx] = 0
        masked_logits = last_token_logits + mask

        # Apply softmax to convert masked logits to probabilities
        masked_probabilities = softmax(masked_logits)
    else:
        masked_probabilities = last_word_probabilities

    # Get the token ID with the highest probability
    most_likely_word_id = masked_probabilities.argmax().item()
    # Decode the token ID to get the word
    predicted_word = tokenizer.decode([most_likely_word_id])

    return predicted_word


# Example usage:

predicted_word = predict_next_word(document[:512] + "Extract from the text what Mrs Braverman said:'", trie,
                                   use_trie=True)
print(predicted_word)

!


In [72]:
# gotta do some proper decoding
# why am i getting ! it should always be one of the words
# maybe there is a need for fine tuning